In [ ]:
"""
        SETUP ENVIRONMENT

Choose a game:
- "Cart" # ok for 100 epoch
- "Car" # not good choice
- "Pendulum" # not good choice
- "Lake" # demand more then 1500 epoch
- "Maze" # 100 epoch is OK
- "Toy" # 100 epoch
"""
GAME_NAME = "Toy"
"""
            INITIALIZING AGENTS and ENVIRONMENT
"""
"""-------------------------------------------------------------------"""
from enviroment import *
env, obs_dim, action_dim = game_setup(GAME_NAME )

In [ ]:
"""
                 SETUP AGENT and TRAINING
"""
"""REINFORCE"""
BETA_REIN_FULL = 0.4 #Betas are scalar by which we multiply bias terms in NN at initialization. They are used to control Chaos order.
BETA_REIN_RESNET = BETA_REIN_FULL
LEARNING_RATE_REIN = 0.0001
HORIZON_REIN = 7
HIDDEN_DIM_REIN = 32
"""MATRYOSHKA"""
BETA_MTR_FULL = 0.1
LEARNING_RATE_MTR = 0.0001
HORIZON_MTR= HORIZON_REIN
HIDDEN_DIM_MTR = 32
TAU = 0.4
"""ResNet"""
BETA_MTR_RESNET = BETA_MTR_FULL
HIDDEN_DIM_RESNET = 16
LEARNING_RATE_RESNET = LEARNING_RATE_MTR * 10
#MTRNet with dynamically changing parameters number per layer
INIT_HIDDEN_LAYER = 10
"""Training"""
NUM_EPOCHES = 500
N_EPISODES = 10 # number of episodes per epoch. Used for both: Reinforce
"""DYNAMICAL TAU AND LR"""
PATIENCE = 20 # for dynamical tau and lr
TAU_END = 0.3 # for dynamical tau
LR_END = LEARNING_RATE_MTR / 1000 # for dynamical learning rate
"""
                    INITIALIZING AGENTS
"""
"""-------------------------------------------------------------------"""
from NeuralNet import ReinforceAgent, MTRAgent
from MtrNet import ReinforceMtrNetAgent, MtrNetAgent, ShortLongAgent
import torch
device = 'cuda' if torch.cuda.device_count() > 0 else 'cpu'
print(device)
#Reinforce Full Connected
agent1 = ReinforceAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_REIN,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_REIN, game_name=GAME_NAME, beta = BETA_MTR_FULL)
agent1.policy.ntk_init(beta=agent1.beta)

#Reinforce MTRNet (Custom ResNet)
agent2 = ReinforceMtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_REIN_RESNET)
agent2.policy.ntk_init(beta=agent2.beta)

#MTR extra-dimension
agent3 = MTRAgent(obs_dim, action_dim, hidden_dim= HIDDEN_DIM_MTR , horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_MTR, game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_FULL)
agent3.policy.ntk_init(beta=agent3.beta)

#MTR with MTRNet
agent4 = MtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET)
agent4.policy.ntk_init(beta=agent4.beta)

#MTR with MTRNet and dynamical number of parameter per layer
agent6 = MtrNetAgent(obs_dim,action_dim, hidden_dim=INIT_HIDDEN_LAYER, horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET, dynamical_layer_param=True)
agent6.policy.ntk_init(beta=agent6.beta)

#ShortLongNet
agent7 = ShortLongAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET)
agent7.policy.ntk_init(beta=agent7.beta)

In [ ]:
"""
        TRAINING (REINFORCE CLASSIC)
        Remark: we are training two reinforce agents
"""
"""-------------------------------------------------------------------"""
from utils import train_agent
loss_list_1 = train_agent(agent1, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END, patience=PATIENCE, clip_grad=False)

In [ ]:
"""
        TRAINING (REINFORCE WITH MTR NN)
        Remark: we are training two reinforce agents
"""
"""-------------------------------------------------------------------"""
from utils import train_agent
loss_list_2 = train_agent(agent2, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END * 10, patience=PATIENCE, clip_grad=False)

In [ ]:
"""
    Matryoshka EXTRA-DIMENSION Training
"""
"""-------------------------------------------------------------------"""
from utils import train_agent
loss_list_3 = train_agent(agent3, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END, patience=PATIENCE, clip_grad=False)

In [ ]:
"""
    MTRNet Training
"""
"""-------------------------------------------------------------------"""
from utils import train_agent
loss_list_4 = train_agent(agent4, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END * 10, patience=PATIENCE, clip_grad=False)

In [ ]:
"""
    MTRNet with dynamical number of parameters Training
"""
"""-------------------------------------------------------------------"""
from utils import train_agent
loss_list_6 = train_agent(agent6, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END * 10, patience=PATIENCE, clip_grad=False)

In [ ]:
"""
    ShortLongNet Training
    Remark: Horizon should be at least 10!
"""
"""-------------------------------------------------------------------"""
from utils import train_agent
agent7.beta = 0.5
agent7.lr = 0.1
agent7.tau = 0.7
agent7.set_optimazer()
agent7.policy.ntk_init(agent7.beta)
loss_list_7 = train_agent(agent7, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END*100 , patience=PATIENCE, clip_grad=False)

In [28]:
"""
    Matryoshka "ORIGINAL" Training
"""
from original import OriginalMtrAgent
# Set-up agent
HIDDEN_DIM_ORIGINAL = 16
HORIZON_ORIGINAL = 7
BETA_ORIGINAL = 0.1
TAU_ORIGINAL = 0.7
LEARNING_RATE_ORIGINAL = LEARNING_RATE_RESNET
"""-------------------------------------------------------------------"""
agent5 = OriginalMtrAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_ORIGINAL,horizon= HORIZON_ORIGINAL, learning_rate= LEARNING_RATE_ORIGINAL , game_name= GAME_NAME, tau=TAU_ORIGINAL, beta=BETA_ORIGINAL)
agent5.policy.ntk_init(beta=agent5.beta)
#training
from utils import train_agent
loss_list_5 = train_agent(agent5, env, num_epoches=NUM_EPOCHES, n_episodes=N_EPISODES, tau_end=TAU_END, lr_end=LR_END, patience=PATIENCE, clip_grad=False)

In [ ]:
"""
        COMPARE REINFORCE VS MTR
        Remark: Run if you have trained both: Reinforce and Matryoshka.
"""
"""-------------------------------------------------------------------"""
fig = plt.figure(figsize=(8,8))
ax1 = fig.add_subplot(1,1,1)
ax1.scatter(range(len(loss_list_1)),loss_list_1, label=f"{agent1.name}: BETA = {agent1.beta}")
ax1.scatter(range(len(loss_list_2)),loss_list_2, label=f"{agent2.name}: BETA = {agent2.beta}")
ax1.scatter(range(len(loss_list_3)),loss_list_3, label=f"{agent3.name}: BETA = {agent3.beta}, TAU = {agent3.tau}")
ax1.scatter(range(len(loss_list_4)),loss_list_4, label=f"{agent4.name}: BETA = {agent4.beta}, TAU = {agent4.tau}")
ax1.set_title(f"Horizon = {HORIZON_MTR}; Game = {GAME_NAME}")
ax1.set_ylabel("Total Reward")
ax1.set_xlabel("Epoch")
ax1.grid()
ax1.legend()
plt.savefig(f'images/{GAME_NAME}_3.jpg', format='jpeg')
plt.show()

In [ ]:
"""
        COMPARE ORIGINAL VS MTRNet VS MTR extra-dim.
        Remark: Run if you have trained both: Original and Matryoshka.
"""
"""-------------------------------------------------------------------"""
fig = plt.figure(figsize=(8,8))
ax1 = fig.add_subplot(1,1,1)
ax1.scatter(range(len(loss_list_5)),loss_list_5, label=f"{agent5.name}: BETA = {agent5.beta}, TAU = {agent5.tau}")
ax1.scatter(range(len(loss_list_3)),loss_list_3, label=f"{agent3.name}: BETA = {agent3.beta}, TAU = {agent3.tau}")
ax1.scatter(range(len(loss_list_4)),loss_list_4, label=f"{agent4.name}: BETA = {agent4.beta}, TAU = {agent4.tau}")
ax1.set_title(f"Horizon = {HORIZON_MTR}; Game = {GAME_NAME}")
ax1.set_ylabel("Total Reward")
ax1.set_xlabel("Epoch")
ax1.grid()
ax1.legend()
plt.savefig(f'images/{GAME_NAME}_6.jpg', format='jpeg')
plt.show()

In [ ]:
"""
        TEST AGENT
"""
NUMBER_OF_EPISODES = 5
"""-------------------------------------------------------------------"""
from enviroment import render
render(agent1,env, NUMBER_OF_EPISODES)